###Abstract ai jobs proyect:
El presente proyecto se centra en el análisis de un dataset de ofertas laborales en el área de IA, que incluye información sobre título del puesto, ubicación, experiencia requerida, tipo de contrato, nivel de trabajo remoto, tamaño de la empresa, habilidades solicitadas, educación requerida, industria, beneficios y salario en dólares estadounidenses.

La elección de este dataset responde al interés de comprender los factores que influyen en la remuneración de profesionales del sector, un área donde las competencias técnicas, la localización y la experiencia pueden tener un peso significativo. El objetivo principal es identificar patrones y relaciones entre las características del puesto y el salario ofrecido, y sentar las bases para un modelo predictivo que permita estimar el salario en función de dichas variables.

El análisis se desarrollará en varias etapas: identificación y tratamiento de valores perdidos, exploración de las distribuciones y relaciones entre variables, creación de visualizaciones multivariadas y cálculo de medidas estadísticas. Este enfoque permitirá responder preguntas clave como:


*   ¿qué nivel de experiencia está mejor remunerado?
*   ¿cuál es la influencia del trabajo remoto?
*   ¿importa más la localización de la empresa o la residencia del empleado?


Los hallazgos obtenidos no solo aportarán información útil para quienes buscan oportunidades laborales en IA, sino también para empresas que desean establecer bandas salariales competitivas. En futuras fases, el proyecto evolucionará hacia la construcción de un modelo para la predicción de salarios, integrando las conclusiones obtenidas en esta etapa inicial.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_validate, GridSearchCV, train_test_split, ValidationCurveDisplay
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

In [ ]:
# Defino el dataframe a partir de la url y separo las features del target
url = 'https://raw.githubusercontent.com/ignaciogualini/Proyecto_Coder_DS1_Gualini/refs/heads/main/datasets/ai_job_dataset.csv'
ai_jobs_df = pd.read_csv(url, index_col='job_id')
target = 'salary_usd'
X, y = (ai_jobs_df.drop(columns=target), ai_jobs_df[target])
ai_jobs_df.head()

### Preguntas de investigación:

1. **¿Cómo afecta la experiencia al salario?**
2. **¿Qué impacto tiene el tamaño de la empresa en el salario?**
3. **¿Existe alguna relación entre el salario y el trabajo remoto?**
4. **¿Por qué algunos salarios son extremadamente altos y qué factores los explican?¿Son outliers?**
---
### Hipótesis:

*  H1. A mayor experiencia, mayor será el salario.
*  H2. Las empresas grandes (`L`) ofrecen salarios más altos que las medianas (`M`) y pequeñas (`S`).
*  H3. Sería de esperar que esta variable (`remote_ratio`) no tenga peso alguno en el salario.
*  H4. Los salarios extremadamente altos corresponden a roles senior o especializados, con muchos años de experiencia y alto nivel de `experience_level`.

In [ ]:
print(f'El dataframe a analizar se compone de {ai_jobs_df.shape[0]} filas y {ai_jobs_df.shape[1]} columnas\n')
print(f'La cantidad de valores nulos por columna es:\n\n {ai_jobs_df.isnull().sum()}')

In [ ]:
# Separo las columnas numéricas de las categóricas
num_col_selector = make_column_selector(dtype_exclude=object)
cat_col_selector = make_column_selector(dtype_include=object) # Use object to include categorical columns
num_cols = num_col_selector(X)
cat_cols = cat_col_selector(X)
X_num = X[num_cols]
X_cat = X[cat_cols]
print(f'De estas {ai_jobs_df.shape[1]} columnas, 1 es el target (salario en USD), {len(num_cols)} son numéricas y {len(cat_cols)} son categóricas')

In [ ]:
print(f'También podemos realizar una breve descripción estadística de sus columnas numéricas:\n\n{X_num.describe()}')
print(f'\nPodemos observar que algunas de las columnas numéricas poseen valores mucho mayores a otras,\npor lo que no es descabellado pensar en una futura estandarización.')
print(f'\nEn cuanto al target:\n\n{y.describe()}')

In [ ]:
# Creo una matriz de correlación para el mapa de calor
correlation = ai_jobs_df[num_cols + [target]].corr()
_ = sns.heatmap(correlation, cmap='Reds', annot=True, linewidths=.5, linecolor='black')
print('Gráfico 1')

Tras realizar un análisis de correlación entre variables numéricas, se puede observar que ninguna de ellas guarda relación más que consigo misma. Por lo que no tenemos columnas numéricas que sean redundantes para el análisis.

También se ve que el target (`salary_usd`) posee una correlación fuerte con los años de experiencia. Por lo que se confirma que es una buena hipótesis a plantear y a resolver en los siguientes gráficos.

In [ ]:
print(f'La variable remote_ratio solo posee {X_num["remote_ratio"].nunique()} valores únicos ({X_num["remote_ratio"].unique()}), por lo que podríamos usar esta informacion en un gráfico.\n')

# Categorizo la variable "company_size" para que aparezca en el orden deseado de tamaño
# X['company_size'] = pd.Categorical(X['company_size'], categories=['S', 'M', 'L'], ordered=True)
print(f'Por otro lado, la variable "company_size" posee los valores {list(X["company_size"].unique())}\n')

print('Gráfico 2.1/Gráfico 2.2')

color_map = {'S': 'black', 'M': 'blue', 'L': 'red'}

# Divido el gráfico en dos ejes separados en dos columnas
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 6))

# Línea de tendencia para ax1
sns.regplot(
    ax=ax1,
    data=ai_jobs_df,
    x='years_experience',
    y=target,
    scatter=False,
    line_kws={'color': 'black'},
    label='línea de tendencia',
    )

# Gráfico de dispersión para ax1
sc1 = ax1.scatter(
    X_num['years_experience'],
    y,
    c=X_num['remote_ratio'],
    cmap='plasma',
    alpha=1,
    edgecolors='w',
    linewidth=0.5,
)
ax1.set_xlabel('Años de experiencia')
ax1.set_ylabel('Salario en USD')
ax1.set_title('Relación Experiencia vs Salario con Remote Ratio')
plt.colorbar(sc1, label='Remote Ratio')
ax1.legend()

# Línea de tendencia para ax2
sns.regplot(
    ax=ax2,
    data=ai_jobs_df,
    x='years_experience',
    y=target,
    scatter=False,
    line_kws={'color': 'black'},
    label='línea de tendencia',
    )

# Gráfico de dispersión para ax2
sc2 = ax2.scatter(
    X_num['years_experience'],
    y,
    c=X['company_size'].map(color_map),
    alpha=0.8
)
ax2.set_xlabel('Años de experiencia')
ax2.set_ylabel('Salario en USD')
ax2.set_title('Relación Experiencia vs Salario con Tamaño de empresa')
plt.colorbar(sc2, label='Company size')
ax2.legend()

plt.tight_layout()
plt.show()

Efectivamente, existe una relación positiva entre los años de experiencia de un trabajador y su salario.
En cuanto a la variable `remote_ratio`, podemos ver que en cada columna del gráfico a la izquierda (2.1) los colores se distribuyen de forma variada, lo cual indica que el Remote Ratio no está tan correlacionado directamente con el salario.

Realizando este mismo análisis en el gráfico de la derecha (2.2), en cada columna se puede observar cierta tendencia: los puntos rojos suelen estar más arriba y los de color negro abajo, por lo cual existe cierta relación positiva entre el salario de un trabajador y el tamaño de la empresa (`company_size`). Entendiendo que mientras más grande sea la empresa, mejor va a pagar a sus empleados.

In [ ]:
variables = ['job_description_length', 'benefits_score', 'years_experience'] # Defino variables de interés para la función de figura
print('Gráfico 3')

# Paso a formato largo para que mi FacetGrid lo entienda
df_long = X.melt(
    id_vars=['company_size'],
    value_vars=variables,
    var_name='variable',
    value_name='valor'
)

# Utilizo una función de figura donde las columnas se representen por el tamaño de la compañía
# En cada fila se encuentran las variables de interés
g = sns.FacetGrid(
    df_long,
    row='variable',
    col='company_size',
    margin_titles=True,
    sharex=False,
    sharey=False
)

g.map(sns.histplot, 'valor', bins=20, color='blue', edgecolor='black', kde=True)

g.set_titles(row_template='{row_name}', col_template='{col_name}')
g.set_axis_labels('', "Frecuencia")
plt.show()

Observando los histogramas, podemos concluir lo siguiente:
la distribución de cada variable numérica (exceptuando "remote ratio") es independiente al tamaño de empresa.
Por lo que estas variables no están sesgadas en función a si una empresa es grande, mediana o chica.

Este tipo de análisis resulta conveniente en estas situaciones, ya que cuando dos variables o features están correlacionadas fuertemente aportan información redundante, y complican la interpretación del modelo.
Un claro ejemplo de esto son las columnas `experience_level` y `years_experience`, ya que apuntan a lo mismo.

In [ ]:
print('Gráfico 4.1/Gráfico 4.2')
sns.set_style('darkgrid')

# Divido el gráfico en dos ejes separados en dos columnas
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(14, 6), sharey=False)

# Elaboro un boxplot para ax1
sns.boxplot(data=ai_jobs_df, y=target, ax=ax1)
ax1.axhline(y=y.mean(), color='red', linestyle='--', linewidth=2, label=f'Salario medio: ${y.mean():.2f}')
ax1.axhline(y=y.mean() - y.std(), color='blue', linestyle='--', linewidth=2, label=f'Límite inferior: ${(y.mean() - y.std()):.2f}')
ax1.axhline(y=y.mean() + y.std(), color='green', linestyle='--', linewidth=2, label=f'Límite superior: ${(y.mean() + y.std()):.2f}')
ax1.set_ylabel('Salario en USD')
ax1.set_title('Boxplot')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Elaboro un histograma para ax2
sns.histplot(data=ai_jobs_df, x=target, ax=ax2, bins=20, edgecolor='black', kde=True)
ax2.axvline(x=y.mean(), color='red', linestyle='--', linewidth=2,)
ax2.axvline(x=y.mean() - y.std(), color='blue', linestyle='--', linewidth=2,)
ax2.axvline(x=y.mean() + y.std(), color='green', linestyle='--', linewidth=2,)
ax2.set_xlabel('Salario en USD')
ax2.set_ylabel('Cantidad')
ax2.set_title('Histograma')
plt.xticks(rotation=45)

_ = plt.suptitle('Distribución de salarios')

plt.tight_layout(rect=[0, 0, 0.95, 1])

plt.show()

print('\nSe puede ver en el gráfico de caja que tenemos muchos valores por fuera de 1.5IQR. Esto puede llegar a indicar que son outliers.')
print('Si analizamos el histograma, vemos que hay una distribución sesgada a la derecha típica de variables salariales:\nla mayoría gana entre un rango bajo/medio, pero hay una minoría con salarios muy altos.')
print('Entonces se puede decir que estos valores son parte natural de la distribución y no un error de registro. Son casos válidos y aportan información relevante para el posterior modelo.')
print(f'Esto también va de la mano con el valor de la desviación estándar: {y.std():.1f} $. En proporción, es el {((y.std() / y.mean()) * 100):.2f} % del salario medio, indicando un valor considerable para este parámetro.\nEsto se relaciona con la distribución sesgada a la derecha, ya que hay muchos valores lejanos a la media.\n')

###Conclusiones preliminares en función de las hipótesis:

*  ***H1. A mayor experiencia, mayor será el salario:***
el mapa de calor (*Gráfico 1*) así como la gráfica de dispersión y la línea de regresión (*Gráficos 2.1 y 2.2*) confirman la relación positiva que guardan estas dos variables, confirmando la hipótesis n° 1✅.

*  ***H2. Las empresas grandes (`L`) ofrecen salarios más altos que las medianas (`M`) y pequeñas (`S`):***
En el segundo gráfico de dispersión (*Gráfico 2.2*) se puede observar que en cada columna los puntos rojos suelen estar más arriba y los de color negro abajo, por lo cual existe cierta relación positiva entre el salario de un trabajador y el tamaño de la empresa. Esto confirma la hipótesis n° 2✅.

*  ***H3. Sería de esperar que esta variable (`remote_ratio`) no tenga peso alguno en el salario:***
Los colores por `remote_ratio` en el primer gráfico de dispersión (*Gráfico 2.1*) no muestran un patrón claro ni correlación fuerte con el salario.
Esto sugiere que el nivel de trabajo remoto no es un factor determinante por sí solo para el salario. Hipótesis confirmada✅.

*  ***H4. Los salarios extremadamente altos corresponden a roles senior o especializados, con muchos años de experiencia y alto nivel de `experience_level`:***
Este apartado va de la mano con la primera hipótesis. En el boxplot (*Gráfico 4.1*) podemos ver que **estadísticamente** tenemos varios outliers. Pero si realizamos un análisis en conjunto con el histograma (*Gráfico 4.2*) los valores altos no son outliers ni errores de medición, son una representación fiel de una distribución sesgada a la derecha. Esta brecha salarial (y a su vez, desviación estándar considerable) es común en este tipo de empleos donde existen muchos factores que generen diferencia en cuanto al pago.


##(i) Feature selection:
En lo que respecta a las variables numéricas, se realizó un análisis preliminar con el objetivo de identificar aquellas que pudieran tener una relación lineal con la variable objetivo (salario). Para ello se construyó un mapa de calor de correlaciones de Pearson, considerando las variables `job_description_length`, `remote_ratio` y `benefits_score`, junto con el target.

Esto permite cuantificar el grado de asociación lineal entre dos variables numéricas, facilitando la detección de posibles redundancias o irrelevancias en el conjunto de datos. En este caso, los valores de correlación obtenidos mostraron una relación débil o prácticamente nula entre dichas variables y el salario, lo que sugiere que no aportan información significativa al modelo predictivo.

En consecuencia, estas variables fueron descartadas del dataset final, con el fin de reducir el ruido, simplificar la dimensionalidad y evitar que características poco informativas afecten el desempeño del modelo.

In [ ]:
print('Gráfico 5')
irrelevant_num_cols = ['job_description_length', 'remote_ratio', 'benefits_score']
correlation = ai_jobs_df[irrelevant_num_cols + [target]].corr()
_ = sns.heatmap(correlation, cmap='Blues', annot=True, linewidths=.5, linecolor='black')

En cuanto a las variables categóricas, también se llevó a cabo un proceso de depuración con el objetivo de reducir ruido y evitar redundancias en el dataset. En primer lugar, se descartaron las variables `salary_currency`, `application_deadline` y `posting_date`, ya que no guardan una relación directa con el salario esperado ni aportan información relevante para la predicción. Este tipo de atributos, al no estar vinculados conceptualmente con la naturaleza del problema, tienden a introducir variabilidad innecesaria y pueden dificultar el aprendizaje del modelo.

Por otro lado, se eliminaron las variables `experience_level` y `required_skills`, dado que su información ya se encuentra representada de forma más detallada en otras características: `years_experience` y `job_title`, respectivamente. Mantener ambas versiones de la misma información habría implicado redundancia y riesgo de colinealidad, lo cual afecta la interpretabilidad y estabilidad de los modelos de regresión.

En síntesis, el descarte de estas variables categóricas respondió a un criterio de relevancia y no redundancia, priorizando aquellas que mejor representan la relación con el target y contribuyen a la construcción de un modelo más robusto y eficiente.

In [ ]:
# Agrupo las features a droppear
drop_cols = ['job_description_length', 'remote_ratio', 'benefits_score','salary_currency', 'application_deadline', 'posting_date', 'experience_level', 'required_skills']
X_fs = X.drop(columns=drop_cols)
X_train, X_test, y_train, y_test = train_test_split(X_fs, y, test_size=0.2, random_state=42)

# Separación de columnas categóricas y numéricas desde el dataset con las características relevantes
num_cols_fs = num_col_selector(X_fs)
cat_cols_fs = cat_col_selector(X_fs)

# Orden jerárquico de las variables categóricas
company_size_order = [["S", "M", "L"]]
education_order = [["Associate", "Bachelor", "Master", "PhD"]]

# Separo variables categóricas en funcion de OneHot o OrdinalEncoder (orden jerárquico)
hierarch_cat = ['company_size', 'education_required']
onehot_cat = [cat for cat in cat_cols_fs if cat not in hierarch_cat]

##(ii) Elección de algoritmo
En esta etapa se procede a la elección del algoritmo de regresión, con el objetivo de capturar de la mejor manera posible la relación entre las variables explicativas y el salario en dólares estadounidenses. En primer lugar, se optó por utilizar Ridge Regression (con y sin estandarización de columnas numéricas), un modelo lineal regularizado que resulta adecuado cuando el dataset contiene un gran número de variables y potencial multicolinealidad entre ellas. La penalización L2 que incorpora este algoritmo ayuda a reducir la varianza y evitar el sobreajuste, manteniendo al mismo tiempo una buena interpretabilidad de los coeficientes.

No obstante, al tratarse de un problema de alta dimensionalidad con interacciones complejas entre variables, también se exploró un enfoque más flexible: la combinación de Nystroem Kernel Approximation con Ridge Regression. Esta técnica permite proyectar los datos a un espacio de mayor complejidad a través de un mapeo no lineal, pero de manera eficiente en términos computacionales gracias a la aproximación de Nystroem. De esta forma, se logra capturar relaciones no lineales en los datos sin recurrir a algoritmos más pesados como Support Vector Regression o redes neuronales.

La comparación entre ambos enfoques permitirá evaluar si el incremento en complejidad (Nystroem + Ridge) se traduce en una mejora significativa en la capacidad predictiva respecto al modelo lineal regularizado (Ridge simple).

Vale aclarar que toda la elección del algoritmo se llevará a cabo con el conjunto de prueba X_train, y_train; reservando el conjunto X_test, y_test para la evaluación del modelo final con distintas métricas.

In [ ]:
# Modelo de Ridge no estandarizado
preproc_non_std = ColumnTransformer(
    transformers=[
    ('hierarch_preproc', OrdinalEncoder(categories=[*company_size_order, *education_order]), hierarch_cat),
    ('onehot_preproc', OneHotEncoder(handle_unknown='ignore'), onehot_cat),
    ('num_preproc', 'passthrough', num_cols_fs)]
)

pipe_non_std = Pipeline(steps=[
    ('preproc', preproc_non_std),
    ('Ridge reg', Ridge())
])

In [ ]:
cv_non_std = cross_validate(
    pipe_non_std,
    X_train,
    y_train,
    cv=10,
    return_estimator=True,
    scoring='neg_mean_absolute_error',
    return_train_score=True,
    error_score='raise',
)
cv_non_std = pd.DataFrame(cv_non_std)
cv_non_std

In [ ]:
# Modelo de Ridge estandarizado
preproc_std = ColumnTransformer(
    transformers=[
    ('hierarch_preproc', OrdinalEncoder(categories=[*company_size_order, *education_order]), hierarch_cat),
    ('onehot_preproc', OneHotEncoder(handle_unknown='ignore'), onehot_cat),
    ('num_preproc', StandardScaler(), num_cols_fs)
])

pipe_std = Pipeline(steps=[
    ('preproc', preproc_std),
    ('Ridge reg', Ridge())
])

In [ ]:
cv_std = cross_validate(
    pipe_std,
    X_train,
    y_train,
    cv=10,
    return_estimator=True,
    scoring='neg_mean_absolute_error',
    return_train_score=True,
    error_score='raise'
)
cv_std = pd.DataFrame(cv_std)
cv_std

In [ ]:
# Modelo de Ridge utilizando Nystroem como aproximación de Kernel proveniente del módulo .kernel_approximation
# El objetivo es generar columnas de grado 3, así el modelo puede captar mejor las relaciones no lineales entre las features. Esto le da más expresividad.
preproc_nyst = ColumnTransformer(transformers=[
    ('hierarch_preproc', OrdinalEncoder(categories=[*company_size_order, *education_order]), hierarch_cat),
    ('onehot_preproc', OneHotEncoder(handle_unknown='ignore'), onehot_cat),
    ('num_preproc', 'passthrough', num_cols_fs)
])
pipe_nystroem = Pipeline(steps=[
    ('preproc', preproc_nyst),
    ('Nystroem', Nystroem(kernel='poly', degree=3, n_components=1000)),
    ('Ridge reg', Ridge())
])

In [ ]:
cv_std_nyst = cross_validate(
    pipe_nystroem,
    X_train,
    y_train,
    cv=10,
    return_estimator=True,
    scoring='neg_mean_absolute_error',
    return_train_score=True,
)
cv_std_nyst = pd.DataFrame(cv_std_nyst)
cv_std_nyst

Posteriormente se procederá a graficar una curva de validación para cada modelo presentado previamente. Se analizará la variación del MAE en función del hiperparámetro en cuestión (alpha).

In [ ]:
param_range = np.logspace(-3, 3, 6)

# Modulo la creación de la curva de validación
def alpha_validation_curve(model, title: str, cv=10):
  disp = ValidationCurveDisplay.from_estimator(
      model,
      X_train,
      y_train,
      cv=cv,
      scoring='neg_mean_absolute_error',
      negate_score=True,
      param_name='Ridge reg__alpha',
      param_range=param_range,
      n_jobs=2,
  )

  disp.ax_.set(xlabel='alpha', ylabel='mean absolute error', title=title)

In [ ]:
print('Gráfico 6.1')
alpha_validation_curve(pipe_non_std, title='Curva de validación de Modelo de Ridge no estandarizado')

In [ ]:
print('Gráfico 6.2')
alpha_validation_curve(pipe_std, title='Curva de validación del Modelo de Ridge estandarizado')

In [ ]:
print('Gráfico 6.3')
alpha_validation_curve(pipe_nystroem, title='Curva de validación del Modelo de Ridge + Nystroem')

Como se puede observar, no existe mucha diferencia entre el Gráfico 6.1 y 6.2. Esto es debido a que no hay necesidad de escalar características numéricas, pues la variable `years_experience` ya está originalmente en una escala comparable a las de las variables categóricas al ser preprocesadas por OneHot o por OrdinalEncoder.
Por otro lado, podemos ver que en la curva de validación perteneciente al último modelo, el error absoluto llega a ser alrededor de casi un 10 % menor en relación a los dos primeros modelos. Esto puede deberse a que, al ser un modelo más expresivo que los anteriores, se captaron relaciones no lineales que se correlacionaban con la variable objetivo `salary_usd`. Podría haberse utilizado también Polynomial Features, pero la dimensionalidad hubiese aumentado, haciendo al modelo costoso computacionalmente hablando.

In [ ]:
# Cuadro resumen de la validación cruzada de 10 pasos para cada modelo
mae_summary = pd.DataFrame({
    'non_std_Ridge': cv_non_std['test_score'],
    'std_Ridge': cv_std['test_score'],
    'std_nystroem_Ridge': cv_std_nyst['test_score']
})
# Append a new row with the mean values using .loc
mae_summary.loc['mean'] = {
    'non_std_Ridge': mae_summary['non_std_Ridge'].mean(),
    'std_Ridge': mae_summary['std_Ridge'].mean(),
    'std_nystroem_Ridge': mae_summary['std_nystroem_Ridge'].mean()
}
mae_summary

Habiendo empleado la curva de validación (variación del hiperparámetro alpha) y el cuadro resumen de la validación cruzada (hiperparámetros fijos), se concluye que el modelo más adecuado para este dataset es el Modelo de Ridge sobre espacio transformado por Nystroem.
Posteriormente se hará un análisis de hiperparámetros para encontrar aquellos que sean más aptos para este caso.

In [ ]:
n_comp_range = np.logspace(2, 3, 5).astype(int)
param_grid = {
    'Ridge reg__alpha': param_range,
    'Nystroem__n_components': n_comp_range
}

model_grid = GridSearchCV(pipe_nystroem, param_grid, cv=10, scoring='neg_mean_absolute_error', n_jobs=-1)
model_grid.fit(X_train, y_train)

In [ ]:
cv_results = pd.DataFrame(model_grid.cv_results_)
cv_results

In [ ]:
# Cuadro resumen del score medio (-MAE) en función de la variación de alpha y del número de componentes del kernel aproximado
col_results = [f'param_{name}' for name in param_grid.keys()]
col_results.append('mean_test_score')
cv_res = pd.DataFrame(model_grid.cv_results_)[col_results]
cv_res.sort_values(by='mean_test_score', ascending=False)

def shorten_name(name):
  if '__' in name:
    return name.split('__')[1]
  return name

cv_res.columns = [shorten_name(col) for col in cv_res.columns]
cv_res

In [ ]:
print('Gráfico 7')
px.parallel_coordinates(
    cv_res.apply({
        'alpha': lambda x: np.log10(x),
        'n_components': lambda x: x,
        'mean_test_score': lambda x: x
    }),
    color='mean_test_score',
    color_continuous_scale=px.colors.sequential.Inferno,
)

En el Gráfico 7 (gráfico de coordenadas paralelas) se puede apreciar la variación de la métrica utilizada para el scoring (-MAE) en relación a los rangos utilizados para ambos hiperparámetros en el GridSearchCV. Es posible interactuar con los ejes para elegir cierto rango y ver qué resultados otorga.

In [ ]:
best_alpha = model_grid.best_params_['Ridge reg__alpha']
best_n_comp = model_grid.best_params_['Nystroem__n_components']
print(f'Los mejores hiperparámetros para el modelo son alpha = {best_alpha:.3f} y n_components = {best_n_comp}')

In [ ]:
best_model = model_grid.best_estimator_
best_model

##(iii) Cálculo de métricas para validar el modelo
A continuación se emplearán distintas métricas para la validación final del modelo. Cabe recordar que estas validaciones se harán con el dataset apartado inicialmente para el test: X_test, y_test.
Las métricas a utilizar son:
* `Error medio absoluto`: mide la magnitud promedio de los errores entre los valores reales y los valores predichos por el modelo, sin considerar su dirección (+/-).
* `Error medio cuadrático`: mide la diferencia al cuadrado entre los valores reales y los valores predichos por el modelo, siendo el promedio de estas diferencias.
* `r2`: indica la proporción de variabilidad en la variable dependiente que puede ser explicada por las variables independientes en el modelo.

In [ ]:
# Se modula la validación cruzada para un modelo y métrica determinado
def cv_metric_test(model, metric: str):

  cv_metric = cross_validate(
      model,
      X_test,
      y_test,
      cv=10,
      scoring=metric,
      n_jobs=-1,
      return_train_score=True,
  )
  return cv_metric['test_score'].mean()

In [ ]:
mae = cv_metric_test(best_model, 'neg_mean_absolute_error')
r2 = cv_metric_test(best_model, 'r2')
mse = cv_metric_test(best_model, 'neg_mean_squared_error')

print(f'El MAE del modelo final es: {-mae:.2f} USD')
print(f'El R2 del modelo final es: {r2:.2f}')
print(f'El MSE del modelo final es: {-mse:.2f} USD^2')

El desempeño del modelo final se evaluó sobre un conjunto de prueba independiente, obteniéndose un MAE de aproximadamente 17.500 USD, un MSE de 607.000.000 USD² y un R² de 0,83. Dado que el salario medio en el dataset es de 115.000 USD, el MAE representa alrededor del 15 % del salario medio, lo que indica que, en promedio, las predicciones se desvían moderadamente del valor real. Por otro lado, la raíz del MSE (RMSE ≈ 24.600 USD) refleja que los errores grandes, influenciados por valores salariales extremos, siguen siendo razonables considerando la dispersión del dataset.

En conjunto, estas métricas muestran que el modelo capta correctamente la mayor parte (83 %) de la variabilidad de los salarios y resulta útil para estimaciones aproximadas de rangos salariales de profesionales de IA. Sin embargo, no se recomienda usarlo para predicciones exactas en casos individuales debido a la presencia de valores atípicos y la variabilidad natural de los salarios en este sector.

## (iv) Conclusiones finales

### Relación entre variables y salario

* Se confirmó que los años de experiencia y el tamaño de la empresa son los factores con mayor influencia sobre el salario, tal como se planteó en las hipótesis iniciales.

* La proporción de trabajo remoto (`remote_ratio`) no mostró un impacto significativo en los salarios, mientras que variables categóricas como `education_required` o `company_size` aportan información útil cuando se codifican de manera jerárquica.

### Selección de características

* Se descartaron variables irrelevantes o redundantes como `job_description_length`, `experience_level`, `required_skills` y fechas de publicación, reduciendo la dimensionalidad y evitando ruido para el modelo.
Esto, a su vez, condujo a la reducción de features y permitió manejar más eficientemente la codificación one-hot, facilitando el entrenamiento de modelos complejos como Ridge con Nystroem.

###Elección de modelos y transformación no lineal

* Se compararon tres enfoques: Ridge lineal sin estandarizar, Ridge lineal estandarizado y Ridge sobre features transformadas por Nystroem.

* La transformación Nystroem permitió capturar relaciones no lineales entre las variables y el salario, logrando la menor magnitud de error (MAE ≈ 17.500 USD) y un R² elevado (≈0,83), mejorando ligeramente el desempeño respecto al Ridge lineal. También se confirmo que no siempre es necesaria una estandarización de datos numéricos.

###Evaluación del desempeño

* Las métricas obtenidas indican que el modelo es capaz de explicar la mayor parte de la variabilidad del salario en el dataset y ofrece estimaciones razonables para rangos salariales.

* La presencia de salarios extremos (outliers) limita la precisión de predicciones individuales, pero estos casos son consistentes con la distribución real del mercado laboral en IA.

###Aplicaciones y utilidad del modelo

* El modelo es útil para estimaciones generales de salarios según experiencia, educación y tamaño de la empresa. Puede servir como referencia para profesionales que buscan oportunidades laborales y para empresas que desean establecer bandas salariales competitivas en roles de IA.

En resumen, el proyecto demuestra que, aun con un dataset con variables categóricas mayoritarias y algunos valores extremos, es posible construir un modelo robusto capaz de capturar patrones relevantes del mercado salarial en IA, y que la combinación de Ridge con Nystroem ofrece una mejora clara al incorporar relaciones no lineales entre las features.